In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
from glob import glob
import yaml
from easydict import EasyDict as edict

In [3]:
import pickle
import os
from os import path
import torch
import numpy as np

from torch_geometric.data import Data

In [4]:
from torch_geometric.utils import dense_to_sparse
from dataset.make_traffic_dataset import TrafficDatasetLoader
from utils.utils import build_fully_connected_edge_idx

In [5]:
from models.GTS.self_attention_graph_learning import Attention_Graph_Learning
from models.GTS.gts_graph_learning import GTS_Graph_Learning

from torch.nn import functional as F


In [6]:
config_file = glob('./config/GTS/metr_la.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [7]:
model = Attention_Graph_Learning(config)

In [8]:
model

Attention_Graph_Learning(
  (attention_gl): GraphLearningMultiHeadAttention(
    (dropout): Dropout(p=0.5, inplace=False)
    (q_layers): ModuleList(
      (0): Linear(in_features=512, out_features=128, bias=False)
      (1): Linear(in_features=512, out_features=128, bias=False)
      (2): Linear(in_features=512, out_features=128, bias=False)
      (3): Linear(in_features=512, out_features=128, bias=False)
    )
    (k_layers): ModuleList(
      (0): Linear(in_features=512, out_features=128, bias=False)
      (1): Linear(in_features=512, out_features=128, bias=False)
      (2): Linear(in_features=512, out_features=128, bias=False)
      (3): Linear(in_features=512, out_features=128, bias=False)
    )
    (attention): GraphLearningScaledDotProductAttention(
      (dropout): Dropout(p=0.4, inplace=False)
      (softmax): Softmax(dim=1)
    )
    (output): Linear(in_features=828, out_features=207, bias=True)
  )
  (fc_concat): Linear(in_features=1472, out_features=512, bias=True)
  (hidde

In [9]:
metr = TrafficDatasetLoader(raw_data_dir='./data/METR-LA', dataset_name='METR-LA')
data_list, X = metr.get_dataset(12, 3, 1)

In [10]:
out = model(X, _)

In [20]:
out.shape

torch.Size([207, 207])

In [11]:
edge_probability = F.gumbel_softmax(out, hard=True)

In [12]:
edge_probability

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<AddBackward0>)

In [21]:
from torch_geometric.utils import to_undirected, to_dense_adj, dense_to_sparse


In [23]:
a, b =dense_to_sparse(edge_probability)

In [25]:
a.shape

torch.Size([2, 207])

In [26]:
b.shape

torch.Size([207])

In [27]:
b

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<IndexBackward0>)